In [6]:
import numpy as np
import pandas as pd
import torch
import nltk
import os.path

from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer

In [30]:
from sentence_transformers import SentenceTransformer
light_model = SentenceTransformer("all-MiniLM-L6-v2")

def get_sentence_embedding(text):
    if isinstance(text,str):
        if not text.strip(): 
            print("Attempted to get embedding for empty text.")
            return []
        
        return light_model.encode(text)
    
    elif isinstance(text,list):
        return [get_sentence_embedding(x) for x in text]
        

/opt/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [23]:
text = "This is a fake review."
text2 = "This is not a fake review."

embedding = get_sentence_embedding(text)
print(type(embedding))

<class 'numpy.ndarray'>


In [24]:
embedding

array([-1.23263024e-01,  5.05560301e-02, -2.25028507e-02,  4.60857414e-02,
        2.99849324e-02, -3.30031365e-02, -1.55494700e-03, -5.69652021e-02,
        5.05095311e-02,  6.48048744e-02,  1.46184955e-03,  3.76170278e-02,
        2.34418996e-02,  6.21915841e-03, -1.52830675e-01,  6.23160042e-03,
        7.50061125e-02, -7.55991414e-02,  1.72824189e-02, -6.93426048e-03,
        3.99190784e-02,  1.52808512e-02,  5.99882342e-02, -6.40675845e-03,
        2.40115095e-02, -3.50265205e-02, -8.36883560e-02,  6.52241185e-02,
       -5.70177026e-02, -3.48392241e-02, -2.79989038e-02,  1.05214700e-01,
       -2.54105385e-02, -3.53250913e-02, -2.24782899e-02,  2.87580397e-02,
        5.49784862e-02,  2.45034229e-02,  1.72102023e-02,  3.67955677e-02,
        7.35448627e-03, -6.07335903e-02,  1.24863116e-02,  3.92360613e-02,
        5.83122000e-02,  7.93062225e-02, -5.64891882e-02, -1.88869201e-02,
        7.02978149e-02,  9.29483771e-02, -2.59401761e-02, -1.08933644e-02,
        4.97389287e-02, -

In [25]:
a = [text, text2]
get_sentence_embedding(a)

[array([-1.23263024e-01,  5.05560301e-02, -2.25028507e-02,  4.60857414e-02,
         2.99849324e-02, -3.30031365e-02, -1.55494700e-03, -5.69652021e-02,
         5.05095311e-02,  6.48048744e-02,  1.46184955e-03,  3.76170278e-02,
         2.34418996e-02,  6.21915841e-03, -1.52830675e-01,  6.23160042e-03,
         7.50061125e-02, -7.55991414e-02,  1.72824189e-02, -6.93426048e-03,
         3.99190784e-02,  1.52808512e-02,  5.99882342e-02, -6.40675845e-03,
         2.40115095e-02, -3.50265205e-02, -8.36883560e-02,  6.52241185e-02,
        -5.70177026e-02, -3.48392241e-02, -2.79989038e-02,  1.05214700e-01,
        -2.54105385e-02, -3.53250913e-02, -2.24782899e-02,  2.87580397e-02,
         5.49784862e-02,  2.45034229e-02,  1.72102023e-02,  3.67955677e-02,
         7.35448627e-03, -6.07335903e-02,  1.24863116e-02,  3.92360613e-02,
         5.83122000e-02,  7.93062225e-02, -5.64891882e-02, -1.88869201e-02,
         7.02978149e-02,  9.29483771e-02, -2.59401761e-02, -1.08933644e-02,
         4.9

In [26]:
df = pd.read_csv("../raw data/combined_data.csv")

In [19]:
df.head()

,Text,Label,Model,Original dataset,Row in original dataset
0,While driverless cars present many promising b...,Machine,darragh_claude_v7,essays,13355
1,Homework Clubs: The Key to Unlocking Academic ...,Machine,llama2_chat,essays,7249
2,"""The legalization of marijuana has been a cont...",Machine,falcon_180b_v1,essays,2603
3,Taking the opportunity to learn new things can...,Machine,mistral7binstruct_v1,essays,3993
4,Working with a partner is an effective way fo...,Machine,mistral7binstruct_v2,essays,3773


In [29]:
tqdm.pandas()
embeddings = df['Text'].progress_apply(get_sentence_embedding)

  0%|          | 0/80000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
df.head()

,Text,Label,Original dataset,Row in original dataset,embedding
0,The idea of graduating high school in three ye...,Machine,essays,26613,"[0.01476596, -0.013095475, 0.002932728, -0.011..."
1,"Hey, I'm so excited to write this essay about ...",Machine,essays,26326,"[0.00292786, -0.013083563, 0.0047025573, -0.00..."
2,Introduction\n\nSelf-reliance is a concept tha...,Machine,essays,30579,"[-0.015756093, -0.023221416, -0.010959062, -0...."
3,"Sure, here's my attempt at writing an essay as...",Machine,essays,33547,"[0.013002162, 0.011017485, -0.03551094, 0.0290..."
4,The legalization of marijuana is a highly deba...,Machine,essays,33768,"[0.0016188276, 0.011155421, -0.004596148, 0.00..."


In [13]:
df.to_csv('combined_data_with_embeddings.csv')

In [18]:
df.to_parquet('combined_data_with_embeddings.parquet')

In [14]:
embeddings = df['embedding']

In [15]:
df['embedding'].apply(lambda a : len(a))

0        1024
1        1024
2        1024
3        1024
4        1024
         ... 
79995    1024
79996    1024
79997    1024
79998    1024
79999    1024
Name: embedding, Length: 80000, dtype: int64

In [16]:
print(sentence_model.max_seq_length)

512
